Problem representation

In [25]:
import numpy as np
M = 9999999

cost = np.array([[7, 5, 5, 0],
                 [3, 10, 10, M],
                 [3, 10, 10, 0],
                 [M, M, 0, 0]])
supply = np.array([30, 20, 80, 80])
demand = np.array([40, 40, 20, 110])






def transportation_simplex(cost, supply, demand):
    m, n = cost.shape
    allocations = np.zeros_like(cost, dtype=float)
    remaining_supply = supply.copy()
    remaining_demand = demand.copy()
    iterations = 0

    valid_cells = np.ones_like(cost, dtype=bool)

    while np.any(remaining_supply > 0) and np.any(remaining_demand > 0):
        iterations += 1

        # Find the minimum cost among valid cells
        min_cost_idx = np.unravel_index(np.argmin(cost * valid_cells), cost.shape)
        i, j = min_cost_idx

        # Allocate as much as possible to the selected cell
        allocation = min(remaining_supply[i], remaining_demand[j])
        allocations[i, j] = allocation
        remaining_supply[i] -= allocation
        remaining_demand[j] -= allocation

        # Update the valid cells to reflect exhausted rows or columns
        if remaining_supply[i] == 0:
            valid_cells[i, :] = False
        if remaining_demand[j] == 0:
            valid_cells[:, j] = False

    return allocations, iterations

allocations, iterations = transportation_simplex(cost, supply, demand)

# Display results
print("\n--- Results ---")
print("Optimal Allocations:")
print(allocations)
print(f"Total Cost: {np.sum(allocations * cost)}")
print(f"Number of Iterations: {iterations}")

KeyboardInterrupt: 

### PuLP verification

In [19]:
def verify_with_pulp(cost, supply, demand):
    prob = pl.LpProblem("Transportation_Problem", pl.LpMinimize)
    rows, cols = cost.shape
    x = pl.LpVariable.dicts("x", (range(rows), range(cols)), 0, None, pl.LpInteger)

    prob += pl.lpSum(cost[i][j] * x[i][j] for i in range(rows) for j in range(cols))

    for i in range(rows):
        prob += pl.lpSum(x[i][j] for j in range(cols)) == supply[i]
    for j in range(cols):
        prob += pl.lpSum(x[i][j] for i in range(rows)) == demand[j]

    prob.solve()

    solution = np.zeros_like(cost, dtype=int)
    for i in range(rows):
        for j in range(cols):
            solution[i, j] = int(x[i][j].varValue)

    return solution, pl.value(prob.objective)

# Example verification:
pulp_solution, pulp_cost = verify_with_pulp(cost, supply, demand)
print("PuLP solution:\n", pulp_solution)
print("PuLP objective value:", pulp_cost)



PuLP solution:
 [[ 0 30  0  0]
 [10 10  0  0]
 [30  0  0 50]
 [ 0  0 20 60]]
PuLP objective value: 370.0
